In [1]:
import pandas as pd

In [40]:
train = pd.read_table("UH_2023_TRAIN.txt", delimiter="|")
train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [14]:
train.shape

(9601, 11)

In [24]:
print("lista de variedad:",list(set(list(train['VARIEDAD'])))) #tipo de variedad
total_variedad = train['VARIEDAD'].value_counts().count() #total tipo de variedad
print("total variedad:", total_variedad)

lista de variedad: [4, 8, 9, 10, 15, 17, 23, 26, 32, 38, 40, 43, 48, 52, 55, 59, 62, 65, 68, 71, 81, 84, 87, 92, 94]
total variedad: 25


In [37]:
print("lista de estacion:",list(set(list(train['ID_ESTACION'])))) #tipo de variedad
total_estacion = train['ID_ESTACION'].value_counts().count() #total tipo de variedad
print("total estacion:", total_estacion)

lista de estacion: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
total estacion: 20


In [34]:
media_produccion = train['PRODUCCION'].mean()
print("Producción media:", round(media_produccion, 2))

Producción media: 9209.19


In [ ]:
matrix = train[,].corr()
print("Correlation matrix is : ")
print(matrix)